In [38]:
import pandas as pd
import numpy as np

In [39]:
# Благо, чтение из файла уже реализовано в test-dataset, возьмем его оттуда
df = pd.read_csv('./data/Run200_Wave_0_1.txt', sep=' ', header=None, skipinitialspace=True)
df = df.drop([0, 1, 2, 3, 504], axis=1)
df.columns = list(range(500))
df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,14820,14823,14824,14822,14818,14820,14824,14822,14820,14820,...,14828,14822,14815,14815,14817,14819,14820,14822,14820,14819
1,14820,14822,14820,14826,14824,14822,14820,14822,14823,14821,...,14828,14817,14824,14822,14824,14819,14820,14819,14822,14820
2,14820,14820,14822,14825,14820,14824,14824,14819,14823,14824,...,14820,14821,14820,14820,14818,14821,14823,14820,14820,14821
3,14828,14822,14818,14824,14824,14822,14820,14822,14824,14820,...,14824,14826,14822,14821,14820,14828,14820,14822,14823,14822
4,14823,14815,14823,14821,14827,14820,14823,14824,14816,14821,...,14820,14820,14823,14828,14824,14820,14824,14824,14822,14825


# Анализ датасета и данных

В первую очередь разберемся, почему в условиях написано, что детектор один, а данных на 500 столбцов. После чтения литературы удалось выяснить, что в сцинтилляционных детекторах 500 значений - это данные с одного детектора, распределенные по времени. Соответственно, один сигнал в 500 значений времени.

Проанализируем датасет со знанием этих вводных

In [40]:
missing_data = df.isnull().sum()

missing_data.loc[missing_data>0]
# Видим, что пропущенных данных нет

Series([], dtype: int64)

Исходя из предположений выше можно говорить о том, что анализировать все 500 столбцов не имеет смысла (т. к. они несут в себе одинаковую информацию).

In [41]:
def find_T0_Tend(samples,
                 N_base=16,
                 thr_sigma=2.5,
                 fixed_window=96):
    """
    samples         — 1D array из 500 ADC-отсчётов (бит)
    N_base          — число точек для оценки базовой линии
    thr_sigma       — порог в σ для детекции T0
    fixed_window    — фиксированная длина окна (ns) для T_end = T0 + fixed_window

    возвращает (T0_idx, T_end_idx) — индексы отсчётов
    """

    # 1) baseline и шум
    base  = samples[:N_base].mean()
    sigma = samples[:N_base].std()

    # 2) порог
    threshold = base + thr_sigma * sigma

    # 3) найти первый отсчёт > threshold
    above = np.nonzero(samples > threshold)[0]
    if len(above)==0:
        # сигнал не вышел из шума
        return None, None
    T0_idx = above[0]

    # 4A) фиксированное окно интегрирования
    T_end_idx = T0_idx + fixed_window
    if T_end_idx > len(samples):
        T_end_idx = len(samples)

    
    return T0_idx, T_end_idx

def calculate_Q_total_PSD(samples, T0, T_end, tail_start=50):
    """
    Улучшенная версия с обработкой крайних случаев корректно:
    если Q_total или Q_tail некорректный — ставим np.nan.
    """
    if T0 is None or T_end is None:
        return np.nan, np.nan, np.nan
    
    T0 = max(0, T0)
    T_end = min(len(samples), T_end)
    
    if T0 >= T_end:
        return np.nan, np.nan, np.nan
    
    # Вычисляем Q_total
    Q_total = np.sum(samples[T0:T_end])
    
    if Q_total <= 0:
        return np.nan, np.nan, np.nan  # нет сигнала вообще
    
    # Определяем границы хвоста
    T_tail_start = T0 + tail_start
    T_tail_start = min(T_tail_start, T_end)  # Не выходим за границу
    T_tail_start = max(T_tail_start, T0)     # Защита от ошибки
    
    # Вычисляем Q_tail
    if T_tail_start < T_end:
        Q_tail = np.sum(samples[T_tail_start:T_end])
    else:
        Q_tail = 0  # фактически хвост отсутствует

    # Проверяем Q_tail
    if Q_tail <= 0:
        return Q_total, np.nan, np.nan  # если хвоста нет, нельзя корректно считать PSD
    
    # Вычисляем PSD
    PSD = Q_tail / Q_total
    
    return Q_total, Q_tail, PSD


# Пример использования с DataFrame
def process_dataframe(df, N_base=16, thr_sigma=2.5, fixed_window=96, tail_start=50):
    """
    Обрабатывает весь DataFrame с сигналами.
    
    Параметры:
        df: DataFrame, где первые 4 столбца - метаданные, остальные 500 - ADC-отсчёты
        N_base, thr_sigma, fixed_window: параметры для find_T0_Tend
        tail_start: параметр для calculate_Q_total_PSD
    
    Возвращает:
        DataFrame с добавленными колонками T0, T_end, Q_total, Q_tail, PSD
    """
    # Выделяем только ADC-отсчёты (предполагаем, что первые 4 колонки - метаданные)
    signals = df.values
    
    # Подготавливаем массивы для результатов
    n_signals = len(signals)
    T0_arr = np.full(n_signals, -1, dtype=int)
    Tend_arr = np.full(n_signals, -1, dtype=int)
    Q_total_arr = np.full(n_signals, np.nan)
    Q_tail_arr = np.full(n_signals, np.nan)
    PSD_arr = np.full(n_signals, np.nan)
    
    # Обработка каждого сигнала
    for i in range(n_signals):
        T0, Tend = find_T0_Tend(signals[i], N_base, thr_sigma, fixed_window)
        
        if T0 is not None:
            T0_arr[i] = T0
            Tend_arr[i] = Tend
            
            Q_total, Q_tail, PSD = calculate_Q_total_PSD(signals[i], T0, Tend, tail_start)
            Q_total_arr[i] = Q_total
            Q_tail_arr[i] = Q_tail
            PSD_arr[i] = PSD
    
    # Добавляем результаты в DataFrame
    result_df = df.copy()
    result_df['T0'] = T0_arr
    result_df['T_end'] = Tend_arr
    result_df['Q_total'] = Q_total_arr
    result_df['Q_tail'] = Q_tail_arr
    result_df['PSD'] = PSD_arr
    
    return result_df

In [42]:
processed_df = process_dataframe(df)


In [49]:
print('Число неклассифицированных сигналов: ', len(processed_df.loc[processed_df['Q_tail'].isna()==True])/len(processed_df))

Число неклассифицированных сигналов:  0.2814429916095234


In [44]:
processed_df.to_csv('processed_data.csv')

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,T0,T_end,Q_total,Q_tail,PSD
36,14825,14824,14821,14824,14821,14822,14825,14819,14823,14824,...,14814,14826,14824,14826,14823,409,500,1348835.0,607716.0,0.450549
69,14822,14823,14827,14821,14824,14828,14827,14825,14820,14820,...,14821,14817,14821,14828,14821,421,500,1170958.0,429852.0,0.367094
84,14820,14823,14823,14823,14823,14828,14824,14820,14821,14827,...,14823,14822,14816,14820,14820,412,500,1304352.0,563230.0,0.431808
90,14817,14824,14822,14821,14822,14825,14820,14814,14820,14823,...,14820,14819,14827,14824,14819,478,500,326096.0,NaN,NaN
105,14824,14821,14823,14823,14818,14821,14820,14826,14822,14824,...,14817,14818,14823,14824,14821,419,500,1200600.0,459489.0,0.382716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23405,14822,14821,14828,14826,14823,14821,14828,14821,14820,14821,...,14823,14820,14829,14818,14825,426,500,1096969.0,355777.0,0.324327
23426,14820,14821,14828,14823,14820,14824,14820,14821,14820,14827,...,14825,14827,14818,14823,14825,474,500,385396.0,NaN,NaN
23441,14827,14818,14826,14823,14824,14824,14823,14824,14829,14822,...,14824,14824,14824,14824,14822,441,500,874564.0,133420.0,0.152556
23448,14821,14828,14826,14822,14821,14824,14829,14824,14821,14820,...,14832,14821,14822,14829,14827,442,500,859803.0,118598.0,0.137936
